# Advanced Prompt Engineering Techniques - Part 1

This notebook demonstrates the first five advanced prompt engineering techniques, including examples, failure cases, and fixes for each approach.

## Table of Contents
1. In-context Learning (ICL)
2. Chain of Thought (CoT)
3. Iterative Chain of Thought (iCoT)
4. Tree of Thought (ToT)
5. Graph of Thought (GoT)

## Setup

In [1]:
!pip install openai
from openai import OpenAI

# Initialize the client
client = OpenAI(api_key="sk-Fake+fakek")

## Helper Functions

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    """Get completion from OpenAI API using the latest client"""
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content

def display_result(prompt, result):
    """Display prompt and its result"""
    print("Prompt:\n", prompt)
    print("\nResult:\n", result)
    print("\n" + "-"*50 + "\n")

def test_prompt(template, test_cases):
    """Test a prompt template with multiple test cases"""
    print("Template:")
    print(template)
    print("\nTest Cases:")
    for case in test_cases:
        print(f"\nInput: {case}")
        prompt = template.format(input=case)
        result = get_completion(prompt)
        print(f"Output: {result}")
        print("-"*50)

## 1. In-context Learning (ICL)

ICL involves providing examples within the prompt to guide the model's responses.

### Success Case

In [3]:
icl_success_prompt = """
Classify the sentiment of movie reviews. Here are some examples:

Review: "This movie was absolutely fantastic! Great acting and storyline."
Sentiment: Positive

Review: "Boring plot, terrible acting, waste of time."
Sentiment: Negative

Review: "The movie was okay, had some good and bad moments."
Sentiment: Neutral

Now classify this review:
Review: "I was blown away by the visual effects and the amazing performance of the lead actor!"
Sentiment:
"""

result = get_completion(icl_success_prompt)
display_result(icl_success_prompt, result)

Prompt:
 
Classify the sentiment of movie reviews. Here are some examples:

Review: "This movie was absolutely fantastic! Great acting and storyline."
Sentiment: Positive

Review: "Boring plot, terrible acting, waste of time."
Sentiment: Negative

Review: "The movie was okay, had some good and bad moments."
Sentiment: Neutral

Now classify this review:
Review: "I was blown away by the visual effects and the amazing performance of the lead actor!"
Sentiment:


Result:
 Positive

--------------------------------------------------



### Failure Case

In [4]:
icl_failure_prompt = """
Review: "The special effects were amazing!"
Rating: 5 stars

Review: "Not worth watching."
Sentiment: Negative

Review: "I was blown away by the visual effects!"
Sentiment:
"""

result = get_completion(icl_failure_prompt)
display_result(icl_failure_prompt, result)

Prompt:
 
Review: "The special effects were amazing!"
Rating: 5 stars

Review: "Not worth watching."
Sentiment: Negative

Review: "I was blown away by the visual effects!"
Sentiment:


Result:
 Positive

--------------------------------------------------



### Fix: Consistent Examples with Clear Format

In [5]:
icl_fix_prompt = """
Task: Classify movie review sentiments as Positive, Negative, or Neutral.

Examples:
Review: "The special effects were amazing and the story was captivating!"
Sentiment: Positive

Review: "Not worth watching, poor acting throughout."
Sentiment: Negative

Review: "It was an average movie, nothing special."
Sentiment: Neutral

Now classify this review:
Review: "I was blown away by the visual effects!"
Sentiment:
"""

result = get_completion(icl_fix_prompt)
display_result(icl_fix_prompt, result)

Prompt:
 
Task: Classify movie review sentiments as Positive, Negative, or Neutral.

Examples:
Review: "The special effects were amazing and the story was captivating!"
Sentiment: Positive

Review: "Not worth watching, poor acting throughout."
Sentiment: Negative

Review: "It was an average movie, nothing special."
Sentiment: Neutral

Now classify this review:
Review: "I was blown away by the visual effects!"
Sentiment:


Result:
 Positive

--------------------------------------------------



## 2. Chain of Thought (CoT)

CoT prompts the model to break down complex problems into intermediate steps.

### Success Case

In [6]:
cot_success_prompt = """
Solve this math word problem step by step:

Example:
Question: A store has 120 apples. They sell 40% of the apples and then receive a new shipment of 30 apples. How many apples do they have now?

Let's solve this step by step:
1. Calculate how many apples were sold:
   * 40% of 120 = (120 × 0.4) = 48 apples sold

2. Calculate remaining apples after sales:
   * 120 - 48 = 72 apples remaining

3. Add new shipment:
   * 72 + 30 = 102 apples total

Therefore, the store now has 102 apples.

Now solve this problem:
Question: A restaurant serves 200 customers per day. If 25% order dessert and each dessert costs $8, how much money does the restaurant make from desserts daily?
"""

result = get_completion(cot_success_prompt)
display_result(cot_success_prompt, result)

Prompt:
 
Solve this math word problem step by step:

Example:
Question: A store has 120 apples. They sell 40% of the apples and then receive a new shipment of 30 apples. How many apples do they have now?

Let's solve this step by step:
1. Calculate how many apples were sold:
   * 40% of 120 = (120 × 0.4) = 48 apples sold

2. Calculate remaining apples after sales:
   * 120 - 48 = 72 apples remaining

3. Add new shipment:
   * 72 + 30 = 102 apples total

Therefore, the store now has 102 apples.

Now solve this problem:
Question: A restaurant serves 200 customers per day. If 25% order dessert and each dessert costs $8, how much money does the restaurant make from desserts daily?


Result:
 Let's solve this step by step:

1. Calculate how many customers order dessert:
   * 25% of 200 = (200 × 0.25) = 50 customers order dessert

2. Calculate the total revenue from desserts:
   * 50 customers × $8 per dessert = $400

Therefore, the restaurant makes $400 from desserts daily.

--------------

### Failure Case

In [7]:
cot_failure_prompt = """
How much money does a restaurant make from desserts if 25% of 200 customers order an $8 dessert?
"""

result = get_completion(cot_failure_prompt)
display_result(cot_failure_prompt, result)

Prompt:
 
How much money does a restaurant make from desserts if 25% of 200 customers order an $8 dessert?


Result:
 The restaurant would make $400 from desserts if 25% of 200 customers order an $8 dessert.

Calculation:
200 customers x 25% = 50 customers
50 customers x $8 = $400

--------------------------------------------------



### Fix: Explicit Step-by-Step Instructions

In [8]:
cot_fix_prompt = """
Calculate the restaurant's daily dessert revenue using these steps:

1. Calculate number of customers who order dessert:
   * Total customers: 200
   * Percentage ordering dessert: 25%

2. Calculate total revenue:
   * Price per dessert: $8
   * Multiply number of dessert orders by price

Show your work for each step.
"""

result = get_completion(cot_fix_prompt)
display_result(cot_fix_prompt, result)

Prompt:
 
Calculate the restaurant's daily dessert revenue using these steps:

1. Calculate number of customers who order dessert:
   * Total customers: 200
   * Percentage ordering dessert: 25%

2. Calculate total revenue:
   * Price per dessert: $8
   * Multiply number of dessert orders by price

Show your work for each step.


Result:
 1. Number of customers who order dessert:
   Total customers: 200
   Percentage ordering dessert: 25%
   Number of customers ordering dessert = 200 * 0.25 = 50 customers

2. Total revenue:
   Price per dessert: $8
   Total revenue = 50 customers * $8 = $400

Therefore, the restaurant's daily dessert revenue is $400.

--------------------------------------------------



## 3. Iterative Chain of Thought (iCoT)

iCoT involves refining the solution through multiple iterations.

### Success Case

In [9]:
icot_success_prompt = """
Design a mobile app feature through multiple iterations:

Feature: User Profile Page

Iteration 1 - Basic Features:
- List essential user information
- Define basic layout
- Identify core functionality

Iteration 2 - Enhancement:
- Add interactive elements
- Improve user experience
- Consider edge cases

Iteration 3 - Polish:
- Add animations
- Optimize performance
- Implement feedback

Provide detailed recommendations for each iteration.
"""

result = get_completion(icot_success_prompt)
display_result(icot_success_prompt, result)

Prompt:
 
Design a mobile app feature through multiple iterations:

Feature: User Profile Page

Iteration 1 - Basic Features:
- List essential user information
- Define basic layout
- Identify core functionality

Iteration 2 - Enhancement:
- Add interactive elements
- Improve user experience
- Consider edge cases

Iteration 3 - Polish:
- Add animations
- Optimize performance
- Implement feedback

Provide detailed recommendations for each iteration.


Result:
 Iteration 1 - Basic Features:
1. List essential user information: Include fields such as name, profile picture, bio, contact information, and any other relevant details that users may want to share on their profile.
2. Define basic layout: Create a clean and organized layout that displays the user information in a visually appealing manner. Consider using tabs or sections to separate different types of information.
3. Identify core functionality: Determine what actions users can take on their profile page, such as editing their in

### Failure Case

In [10]:
icot_failure_prompt = """
Design a user profile page for a mobile app.
"""

result = get_completion(icot_failure_prompt)
display_result(icot_failure_prompt, result)

Prompt:
 
Design a user profile page for a mobile app.


Result:
 User Profile Page

[Profile Picture]

[Username]

[Edit Profile Button]

[Personal Information]
- Name
- Email
- Phone Number
- Date of Birth

[Address]
- Street Address
- City
- State
- Zip Code

[Preferences]
- Favorite Categories
- Notification Settings

[Activity]
- Recent Activity
- Points Earned
- Badges Earned

[Logout Button]

--------------------------------------------------



### Fix: Structured Iterations with Context

In [11]:
icot_fix_prompt = """
Design a user profile page through these specific iterations:

Context:
- Platform: iOS and Android
- Target users: Young professionals
- Key requirement: Professional networking

Iteration 1 - Core Features:
- Essential information display
   * Profile picture
   * Basic info
   * Professional summary
- Basic layout structure
- Navigation elements

Iteration 2 - User Experience:
- Interactive elements
   * Edit capabilities
   * Section expansion
   * Quick actions
- Visual hierarchy
- Accessibility features

Iteration 3 - Polish & Performance:
- Animations & transitions
- Loading states
- Error handling
- Performance optimization

For each iteration:
1. List specific features
2. Explain implementation approach
3. Consider potential challenges
"""

result = get_completion(icot_fix_prompt)
display_result(icot_fix_prompt, result)

Prompt:
 
Design a user profile page through these specific iterations:

Context:
- Platform: iOS and Android
- Target users: Young professionals
- Key requirement: Professional networking

Iteration 1 - Core Features:
- Essential information display
   * Profile picture
   * Basic info
   * Professional summary
- Basic layout structure
- Navigation elements

Iteration 2 - User Experience:
- Interactive elements
   * Edit capabilities
   * Section expansion
   * Quick actions
- Visual hierarchy
- Accessibility features

Iteration 3 - Polish & Performance:
- Animations & transitions
- Loading states
- Error handling
- Performance optimization

For each iteration:
1. List specific features
2. Explain implementation approach
3. Consider potential challenges


Result:
 Iteration 1 - Core Features:
1. Essential information display:
   - Profile picture: Allow users to upload a profile picture or choose from pre-set avatars.
   - Basic info: Name, job title, company, location.
   - Professio

## 4. Tree of Thought (ToT)

ToT explores multiple solution paths simultaneously.

### Success Case

In [12]:
tot_success_prompt = """
Choose the best technology stack for a new e-commerce platform by exploring different paths:

Path 1: MERN Stack
- MongoDB for flexible data
- Express.js for API
- React for frontend
- Node.js for backend

Path 2: Python Stack
- PostgreSQL for data
- Django for backend
- React for frontend
- Redis for caching

Path 3: Java Stack
- MySQL for data
- Spring Boot for backend
- Angular for frontend
- Elasticsearch for search

Requirements:
- High scalability
- Real-time features
- SEO friendly
- Easy maintenance

Analyze each path and recommend the best option.
"""

result = get_completion(tot_success_prompt)
display_result(tot_success_prompt, result)

Prompt:
 
Choose the best technology stack for a new e-commerce platform by exploring different paths:

Path 1: MERN Stack
- MongoDB for flexible data
- Express.js for API
- React for frontend
- Node.js for backend

Path 2: Python Stack
- PostgreSQL for data
- Django for backend
- React for frontend
- Redis for caching

Path 3: Java Stack
- MySQL for data
- Spring Boot for backend
- Angular for frontend
- Elasticsearch for search

Requirements:
- High scalability
- Real-time features
- SEO friendly
- Easy maintenance

Analyze each path and recommend the best option.


Result:
 Path 1: MERN Stack
- MongoDB is known for its flexibility and scalability, making it a good choice for an e-commerce platform that requires high scalability.
- Express.js and Node.js are known for their real-time features, making them suitable for real-time features in an e-commerce platform.
- React is SEO friendly and provides a great user experience for the frontend.
- Node.js is easy to maintain and has a lar

### Failure Case

In [13]:
tot_failure_prompt = """
What's the best technology stack for an e-commerce website?
"""

result = get_completion(tot_failure_prompt)
display_result(tot_failure_prompt, result)

Prompt:
 
What's the best technology stack for an e-commerce website?


Result:
 The best technology stack for an e-commerce website will depend on various factors such as the size of the website, the expected traffic, the budget, and the specific requirements of the business. However, some commonly used technology stacks for e-commerce websites include:

1. Frontend: React.js or Angular for building dynamic and interactive user interfaces.
2. Backend: Node.js or Python with Django for building scalable and robust backend systems.
3. Database: MongoDB or MySQL for storing and managing product data, user information, and transactions.
4. Server: AWS or Google Cloud for hosting the website and ensuring high availability and performance.
5. Payment Gateway: Stripe or PayPal for secure and seamless payment processing.
6. Security: SSL certificates, encryption, and regular security audits to protect customer data and prevent cyber attacks.

Ultimately, the best technology stack for an e-com

### Fix: Structured Decision Tree with Evaluation Criteria

In [14]:
tot_fix_prompt = """
Evaluate technology stacks for an e-commerce platform using this decision tree:

Project Context:
- Scale: 100,000 monthly users
- Budget: Medium
- Timeline: 6 months
- Team: 5 developers

Evaluation Criteria:

1. Frontend Framework
   ├─ React
   │  ├─ Component reusability
   │  └─ Performance optimization
   ├─ Angular
   │  ├─ Enterprise features
   │  └─ TypeScript integration
   └─ Vue.js
      ├─ Learning curve
      └─ Ecosystem maturity

2. Backend Technology
   ├─ Node.js
   ├─ Django
   └─ Spring Boot

3. Database
   ├─ PostgreSQL
   ├─ MongoDB
   └─ MySQL

4. Additional Services
   ├─ Caching (Redis)
   ├─ Search (Elasticsearch)
   └─ Queue (RabbitMQ)

For each branch:
1. Evaluate technical fit
2. Consider team expertise
3. Assess maintenance needs
4. Analyze scaling capabilities

Provide a final recommendation with justification.
"""

result = get_completion(tot_fix_prompt)
display_result(tot_fix_prompt, result)

Prompt:
 
Evaluate technology stacks for an e-commerce platform using this decision tree:

Project Context:
- Scale: 100,000 monthly users
- Budget: Medium
- Timeline: 6 months
- Team: 5 developers

Evaluation Criteria:

1. Frontend Framework
   ├─ React
   │  ├─ Component reusability
   │  └─ Performance optimization
   ├─ Angular
   │  ├─ Enterprise features
   │  └─ TypeScript integration
   └─ Vue.js
      ├─ Learning curve
      └─ Ecosystem maturity

2. Backend Technology
   ├─ Node.js
   ├─ Django
   └─ Spring Boot

3. Database
   ├─ PostgreSQL
   ├─ MongoDB
   └─ MySQL

4. Additional Services
   ├─ Caching (Redis)
   ├─ Search (Elasticsearch)
   └─ Queue (RabbitMQ)

For each branch:
1. Evaluate technical fit
2. Consider team expertise
3. Assess maintenance needs
4. Analyze scaling capabilities

Provide a final recommendation with justification.


Result:
 1. Frontend Framework:
- React: Fits well technically, good for component reusability and performance optimization. Team exp

## 5. Graph of Thought (GoT)

GoT represents problem-solving as an interconnected network of concepts.

### Success Case

In [15]:
got_success_prompt = """
Analyze the system architecture for a social media platform:

Components (Nodes):
1. User Interface (React)
   - News Feed
   - Profile Page
   - Messaging

2. API Layer (GraphQL)
   - Query Resolution
   - Data Aggregation
   - Rate Limiting

3. Services
   - Authentication
   - Content Management
   - Notification

4. Data Storage
   - User Data (PostgreSQL)
   - Media (S3)
   - Cache (Redis)

Relationships (Edges):
- UI → API: REST/GraphQL calls
- API → Services: Internal communication
- Services → Storage: Data operations
- Services ↔ Cache: Performance optimization

For each component:
1. Analyze its responsibilities
2. Identify dependencies
3. Suggest optimizations
4. Consider scaling challenges
"""

result = get_completion(got_success_prompt)
display_result(got_success_prompt, result)

Prompt:
 
Analyze the system architecture for a social media platform:

Components (Nodes):
1. User Interface (React)
   - News Feed
   - Profile Page
   - Messaging

2. API Layer (GraphQL)
   - Query Resolution
   - Data Aggregation
   - Rate Limiting

3. Services
   - Authentication
   - Content Management
   - Notification

4. Data Storage
   - User Data (PostgreSQL)
   - Media (S3)
   - Cache (Redis)

Relationships (Edges):
- UI → API: REST/GraphQL calls
- API → Services: Internal communication
- Services → Storage: Data operations
- Services ↔ Cache: Performance optimization

For each component:
1. Analyze its responsibilities
2. Identify dependencies
3. Suggest optimizations
4. Consider scaling challenges


Result:
 1. User Interface:
   - Responsibilities: Displaying the news feed, profile page, and messaging features to users.
   - Dependencies: Relies on API calls to fetch data and interact with the backend services.
   - Optimizations: Implement client-side caching to reduce 

### Failure Case

In [16]:
got_failure_prompt = """
How should I design a social media platform?
"""

result = get_completion(got_failure_prompt)
display_result(got_failure_prompt, result)

Prompt:
 
How should I design a social media platform?


Result:
 Designing a social media platform involves several key considerations to ensure a user-friendly and engaging experience for your users. Here are some steps to consider when designing a social media platform:

1. Define your target audience: Understand who your target users are and what their needs and preferences are. This will help you tailor the platform to meet their specific requirements.

2. Choose the right features: Determine what features you want to include on your platform, such as user profiles, news feeds, messaging, groups, events, and more. Make sure these features align with the needs and interests of your target audience.

3. Create a user-friendly interface: Design a clean and intuitive interface that is easy to navigate and visually appealing. Consider using a responsive design to ensure your platform is accessible on different devices.

4. Focus on user engagement: Implement features that encourage use

### Fix: Detailed Component Graph with Analysis Framework

In [17]:
got_fix_prompt = """
Design a social media platform architecture using this component graph:

A. Frontend Layer
   Components:
   - User Interface (React)
     * News Feed Component
     * Profile Component
     * Messaging Component
   - State Management (Redux)
   - Real-time Updates (WebSocket)

B. API Gateway Layer
   Components:
   - GraphQL Server
     * Query Resolution
     * Data Aggregation
   - Authentication Middleware
   - Rate Limiting

C. Service Layer
   Components:
   - User Service
     * Profile Management
     * Authentication
   - Content Service
     * Post Management
     * Media Processing
   - Messaging Service
     * Chat Management
     * Notifications

D. Data Layer
   Components:
   - Primary Database (PostgreSQL)
     * User Data
     * Content Data
   - Cache Layer (Redis)
     * Session Data
     * Frequent Queries
   - File Storage (S3)
     * Media Files
     * Backups

Connections:
1. A → B: GraphQL/REST API calls
2. B → C: Internal service communication
3. C → D: Data operations
4. A ↔ C: WebSocket connections
5. B ↔ D: Cache operations

Analysis Framework:
For each component:
1. Core Responsibilities
   - Primary functions
   - Data handling
   - Error management

2. Dependencies
   - Required services
   - External systems
   - Communication patterns

3. Performance Considerations
   - Scaling strategy
   - Caching approach
   - Optimization opportunities

4. Security Measures
   - Authentication
   - Authorization
   - Data protection

Provide a comprehensive analysis following this framework.
"""

result = get_completion(got_fix_prompt)
display_result(got_fix_prompt, result)

Prompt:
 
Design a social media platform architecture using this component graph:

A. Frontend Layer
   Components:
   - User Interface (React)
     * News Feed Component
     * Profile Component
     * Messaging Component
   - State Management (Redux)
   - Real-time Updates (WebSocket)

B. API Gateway Layer
   Components:
   - GraphQL Server
     * Query Resolution
     * Data Aggregation
   - Authentication Middleware
   - Rate Limiting

C. Service Layer
   Components:
   - User Service
     * Profile Management
     * Authentication
   - Content Service
     * Post Management
     * Media Processing
   - Messaging Service
     * Chat Management
     * Notifications

D. Data Layer
   Components:
   - Primary Database (PostgreSQL)
     * User Data
     * Content Data
   - Cache Layer (Redis)
     * Session Data
     * Frequent Queries
   - File Storage (S3)
     * Media Files
     * Backups

Connections:
1. A → B: GraphQL/REST API calls
2. B → C: Internal service communication
3. C → 